# SIR epidemic model without vital dynamics

In [1]:
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CrosshairTool, HoverTool
from scipy.integrate import odeint, solve_ivp

output_notebook()

Loading BokehJS ...

<div class="admonition note">
  <p class="admonition-title">About the code for plots and tables</p>
  <p>
      The code for the plots and some tables is hidden on purpose.
      This is to avoid repeating code. Go to the repo to find the code.
  </p>
</div>

## Introduction

Susceptible-Infectious-Recovered (SIR) model is an example of a compartmental model that is widely used to forecast the progress of disease epidemics.

A simple mathematical description of the spread of a disease in a population is the so-called SIR model, which divides the (fixed) population of **N** individuals into three categories which vary as a function of time, **t**:

* S(t) are those susceptible but not yet infected with the disease;
* I(t) is the number of infectious individuals;
* R(t) are those individuals who have recovered from the disease and now have immunity to it.


The SIR model describes the change in the population of each of these compartments in terms of two parameters, β and γ
* β  
    describes the effective contact rate of the disease: an infected individual comes into contact with βN  
    other individuals per unit time (of which the fraction that are susceptible to contracting the disease is S/N)
* γ  
    is the mean recovery rate: that is, 1/γ  
    is the mean period of time during which an infected individual can pass it on.

The differential equations describing this model were first derived by Kermack and McKendrick [Proc. R. Soc. A, 115, 772 (1927)]:

$$
    \frac{dS}{dt}\ = -\frac{βSI}{N},
$$

$$
    \frac{dI}{dt}\ = -\frac{βSI}{N}-γI,
$$

$$
    \frac{dR}{dt}\ = -γI
$$

Basic reproduction number or average number of people infected from one other person. If it is high, the probability of pandemic is also high:
$$
    R_{0} = \frac{β}{γ}\
$$

## Basic Simulation

In [2]:
# Total population, N.
N = 1000
# Initial number of infected and recovered individuals, I0 and R0.
I0, R0 = 1, 0
# Starting with 1 infected
I0, R0 = 1, 0
# Everyone else, S0, is susceptible to infection initially.
S0 = N - I0 - R0
# Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
beta, gamma = 0.2, 1./10 
# A grid of time points (in days)
t = np.linspace(0, 160, 161)

# The SIR model differential equations.
def deriv(t, y, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return [dSdt, dIdt, dRdt]

# Initial conditions vector
y0 = S0, I0, R0

# Integrate the SIR equations over the time grid, t.
result = solve_ivp(fun=deriv, t_span=[t[0], t[-1]], y0=y0, t_eval=t, args=(N, beta, gamma))
# S, I, R = ret.T

### Result

In [3]:
source = ColumnDataSource(data={
    'time': result.t,
    'susceptible': result.y[0]/N,
    'infected': result.y[1]/N,
    'recovered': result.y[2]/N
})

fig = figure(sizing_mode='stretch_width', x_axis_label='Time (days)',
             y_axis_label='Number of people', toolbar_location='above',
             height=500, max_width=1000)

fig.line(x='time', y='susceptible', legend_label='Susceptible',
         color='blue', source=source)
fig.line(x='time', y='infected', legend_label='Infected',
         color='red', source=source)
fig.line(x='time', y='recovered', legend_label='Recovered with immunity',
         color='green',source=source)

fig.grid.visible = False
fig.toolbar.autohide = True
fig.legend.location = 'top_right'
fig.legend.click_policy= 'hide'

fig.add_tools(CrosshairTool(dimensions='height'))
tooltips = [
    ('Time', '@time'),
    ('N people', '$snap_y{0.00%}'),
    # ('price', '@prices_exc{0.00}€ - @prices_acc{$0.00}')
]
fig.add_tools(HoverTool(tooltips=tooltips, mode='vline'))

show(fig)

## References

- M. J. Keeling and P. Rohani, [Modeling Infectious Diseases in Humans and Animals](https://press.princeton.edu/books/hardcover/9780691116174/modeling-infectious-diseases-in-humans-and-animals), Princeton (2007).
- R. M. Anderson and R. M. May, [Infectious Diseases of Humans: Dynamics and Control](https://www.amazon.co.uk/Infectious-Diseases-Humans-Dynamics-Publications/dp/019854040X), OUP (1992).

## Further Reading
- https://www.lewuathe.com/covid-19-dynamics-with-sir-model.htmlhttps://www.lewuathe.com/covid-19-dynamics-with-sir-model.html